# Trainer

> This module contains code to build custom HuggingFace Trainer

In [ ]:
#| default_exp trainers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
from transformers import Trainer,get_cosine_with_hard_restarts_schedule_with_warmup
from torch.optim import AdamW

In [ ]:
#| export
def get_cosine_restart_class(warmup_ratio=0.1,num_cycles=2):
    """
    Class getter for a Trainer that consists of Cosine Restart with Muptiple Cycles LR Scheduler
    
    Source: https://discuss.huggingface.co/t/how-do-use-lr-scheduler/4046/8
    """
    class CustomTrainer(Trainer):
        def __init__(self, *args, **kwargs):
            self.warmup_ratio=warmup_ratio
            self.num_cycles=num_cycles
            super().__init__(*args, **kwargs)

        def create_optimizer_and_scheduler(self, num_training_steps):
            self.optimizer = AdamW(self.model.parameters(),
                                   lr=self.args.learning_rate,
                                   weight_decay=self.args.weight_decay)
            self.lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
                 self.optimizer, int(self.warmup_ratio*num_training_steps), num_training_steps, num_cycles=self.num_cycles)    
    return CustomTrainer

This can be used as a Trainer for any tasks, either supervised (classification/regression/multilabel ...) or semi-supervised (language model) on any model, either vanilla or customed

Example: Training a `RobertaHiddenStateConcatForSequenceClassification` model with Cosine Restart LR scheduler for **2 cycles** with **warmup ratio of 0.01**

```python3
model_name='roberta-base'
roberta_body = RobertaModel.from_pretrained(model_name)

model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                  cpoint_path = model_name, 
                                  output_hidden_states=True,
                                  seed=seed,
                                  body_model=roberta_body,
                                  model_kwargs = _model_kwargs)
controller = ModelController(model,tdc,seed=seed)

metric_funcs = partial(f1_score,average='macro')
trainer_class = get_cosine_restart_class(warmup_ratio=0.01,num_cycles=2)

controller.fit(epochs=8,
               learning_rate=1e-4,
               batch_size=64,
               save_checkpoint=False,
               metric_funcs=metric_funcs,
               trainer_class=trainer_class,
               compute_metrics=compute_metrics,
              )
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()